# Colab FAQ

For some basic overview and features offered in Colab notebooks, check out: [Overview of Colaboratory Features](https://colab.research.google.com/notebooks/basic_features_overview.ipynb)

You need to use the colab GPU for this assignmentby selecting:

> **Runtime**   →   **Change runtime type**   →   **Hardware Accelerator: GPU**

## Setup PyTorch
All files are stored at /content/csc421/a4/ folder


In [1]:
######################################################################
# Setup python environment and change the current working directory
######################################################################
!pip install torch torchvision
!pip install imageio

!pip install matplotlib

%mkdir -p /content/csc413/a4/
%cd /content/csc413/a4

/content/csc413/a4


# Helper code

## Utility functions

In [2]:
import os

import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.nn import Parameter
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms

from six.moves.urllib.request import urlretrieve
import tarfile

import imageio
from urllib.error import URLError
from urllib.error import HTTPError


def get_file(fname,
             origin,
             untar=False,
             extract=False,
             archive_format='auto',
             cache_dir='data'):
    datadir = os.path.join(cache_dir)
    if not os.path.exists(datadir):
        os.makedirs(datadir)

    if untar:
        untar_fpath = os.path.join(datadir, fname)
        fpath = untar_fpath + '.tar.gz'
    else:
        fpath = os.path.join(datadir, fname)

    print(fpath)
    if not os.path.exists(fpath):
        print('Downloading data from', origin)

        error_msg = 'URL fetch failure on {}: {} -- {}'
        try:
            try:
                urlretrieve(origin, fpath)
            except URLError as e:
                raise Exception(error_msg.format(origin, e.errno, e.reason))
            except HTTPError as e:
                raise Exception(error_msg.format(origin, e.code, e.msg))
        except (Exception, KeyboardInterrupt) as e:
            if os.path.exists(fpath):
                os.remove(fpath)
            raise

    if untar:
        if not os.path.exists(untar_fpath):
            print('Extracting file.')
            with tarfile.open(fpath) as archive:
                archive.extractall(datadir)
        return untar_fpath

    return fpath


class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

                
def to_var(tensor, cuda=True):
    """Wraps a Tensor in a Variable, optionally placing it on the GPU.

        Arguments:
            tensor: A Tensor object.
            cuda: A boolean flag indicating whether to use the GPU.

        Returns:
            A Variable object, on the GPU if cuda==True.
    """
    if cuda:
        return Variable(tensor.cuda())
    else:
        return Variable(tensor)

    
def to_data(x):
    """Converts variable to numpy."""
    if torch.cuda.is_available():
        x = x.cpu()
    return x.data.numpy()


def create_dir(directory):
    """Creates a directory if it doesn't already exist.
    """
    if not os.path.exists(directory):
        os.makedirs(directory)


def gan_checkpoint(iteration, G, D, opts):
    """Saves the parameters of the generator G and discriminator D.
    """
    G_path = os.path.join(opts.checkpoint_dir, 'G.pkl')
    D_path = os.path.join(opts.checkpoint_dir, 'D.pkl')
    torch.save(G.state_dict(), G_path)
    torch.save(D.state_dict(), D_path)

def load_checkpoint(opts):
    """Loads the generator and discriminator models from checkpoints.
    """
    G_path = os.path.join(opts.load, 'G.pkl')
    D_path = os.path.join(opts.load, 'D_.pkl')

    G = DCGenerator(noise_size=opts.noise_size, conv_dim=opts.g_conv_dim, spectral_norm=opts.spectral_norm)
    D = DCDiscriminator(conv_dim=opts.d_conv_dim)

    G.load_state_dict(torch.load(G_path, map_location=lambda storage, loc: storage))
    D.load_state_dict(torch.load(D_path, map_location=lambda storage, loc: storage))

    if torch.cuda.is_available():
        G.cuda()
        D.cuda()
        print('Models moved to GPU.')

    return G, D


def merge_images(sources, targets, opts):
    """Creates a grid consisting of pairs of columns, where the first column in
    each pair contains images source images and the second column in each pair
    contains images generated by the CycleGAN from the corresponding images in
    the first column.
    """
    _, _, h, w = sources.shape
    row = int(np.sqrt(opts.batch_size))
    merged = np.zeros([3, row * h, row * w * 2])
    for (idx, s, t) in (zip(range(row ** 2), sources, targets, )):
        i = idx // row
        j = idx % row
        merged[:, i * h:(i + 1) * h, (j * 2) * h:(j * 2 + 1) * h] = s
        merged[:, i * h:(i + 1) * h, (j * 2 + 1) * h:(j * 2 + 2) * h] = t
    return merged.transpose(1, 2, 0)


def generate_gif(directory_path, keyword=None):
    images = []
    for filename in sorted(os.listdir(directory_path)):
        if filename.endswith(".png") and (keyword is None or keyword in filename):
            img_path = os.path.join(directory_path, filename)
            print("adding image {}".format(img_path))
            images.append(imageio.imread(img_path))

    if keyword:
        imageio.mimsave(
            os.path.join(directory_path, 'anim_{}.gif'.format(keyword)), images)
    else:
        imageio.mimsave(os.path.join(directory_path, 'anim.gif'), images)


def create_image_grid(array, ncols=None):
    """
    """
    num_images, channels, cell_h, cell_w = array.shape
    if not ncols:
        ncols = int(np.sqrt(num_images))
    nrows = int(np.math.floor(num_images / float(ncols)))
    result = np.zeros((cell_h * nrows, cell_w * ncols, channels), dtype=array.dtype)
    for i in range(0, nrows):
        for j in range(0, ncols):
            result[i * cell_h:(i + 1) * cell_h, j * cell_w:(j + 1) * cell_w, :] = array[i * ncols + j].transpose(1, 2,
                                                                                                                 0)

    if channels == 1:
        result = result.squeeze()
    return result


def gan_save_samples(G, fixed_noise, iteration, opts):
    generated_images = G(fixed_noise)
    generated_images = to_data(generated_images)

    grid = create_image_grid(generated_images)

    # merged = merge_images(X, fake_Y, opts)
    path = os.path.join(opts.sample_dir, 'sample-{:06d}.png'.format(iteration))
    imageio.imwrite(path, grid)
    print('Saved {}'.format(path))

## Data loader

In [3]:
def get_emoji_loader(emoji_type, opts):
    """Creates training and test data loaders.
    """
    transform = transforms.Compose([
                    transforms.Scale(opts.image_size),
                    transforms.ToTensor(),
                    transforms.Normalize((0.5, ), (0.5, )),
                ])

    train_path = os.path.join('data/emojis', emoji_type)
    test_path = os.path.join('data/emojis', 'Test_{}'.format(emoji_type))

    train_dataset = datasets.ImageFolder(train_path, transform)
    test_dataset = datasets.ImageFolder(test_path, transform)


    train_dloader = DataLoader(dataset=train_dataset, batch_size=opts.batch_size, shuffle=True, num_workers=opts.num_workers)
    test_dloader = DataLoader(dataset=test_dataset, batch_size=opts.batch_size, shuffle=False, num_workers=opts.num_workers)

    return train_dloader, test_dloader

def get_emnist_loader(emnist_type, opts):
    transform = transforms.Compose([
                    transforms.Scale(opts.image_size),
                    transforms.ToTensor(),
                    transforms.Normalize((0.5), (0.5)),
                ])
    train = datasets.EMNIST(".", split=emnist_type,train = True, download = True, transform= transform)
    test = datasets.EMNIST(".", split=emnist_type,train = False, download = True, transform = transform)
    
    train_dloader = DataLoader(dataset=train, batch_size=opts.batch_size, shuffle=True,num_workers=opts.num_workers)
    test_dloader = DataLoader(dataset=test, batch_size=opts.batch_size, shuffle=False,num_workers=opts.num_workers)
    return train_dloader, test_dloader





## Training and evaluation code

In [4]:
def print_models(G_XtoY, G_YtoX, D_X, D_Y):
    """Prints model information for the generators and discriminators.
    """
    print("                 G                     ")
    print("---------------------------------------")
    print(G_XtoY)
    print("---------------------------------------")

    print("                  D                    ")
    print("---------------------------------------")
    print(D_X)
    print("---------------------------------------")


def create_model(opts):
    """Builds the generators and discriminators.
    """
    ### GAN
    G = DCGenerator(noise_size=opts.noise_size, conv_dim=opts.g_conv_dim, spectral_norm=opts.spectral_norm)
    D = DCDiscriminator(conv_dim=opts.d_conv_dim, spectral_norm=opts.spectral_norm)

    print_models(G, None, D, None)

    if torch.cuda.is_available():
        G.cuda()
        D.cuda()
        print('Models moved to GPU.')
    return G, D

def train(opts):
    """Loads the data, creates checkpoint and sample directories, and starts the training loop.
    """

    # Create train and test dataloaders for images from the two domains X and Y
    dataloader_X, test_dataloader_X = get_emnist_loader(opts.X, opts=opts)
    
    # Create checkpoint and sample directories
    create_dir(opts.checkpoint_dir)
    create_dir(opts.sample_dir)

    # Start training
    G, D = gan_training_loop(dataloader_X, test_dataloader_X, opts)
    return G, D

def print_opts(opts):
    """Prints the values of all command-line arguments.
    """
    print('=' * 80)
    print('Opts'.center(80))
    print('-' * 80)
    for key in opts.__dict__:
        if opts.__dict__[key]:
            print('{:>30}: {:<30}'.format(key, opts.__dict__[key]).center(80))
    print('=' * 80)


# Your code for generators and discriminators

## Helper modules

In [5]:
def sample_noise(batch_size, dim):
    """
    Generate a PyTorch Tensor of uniform random noise.

    Input:
    - batch_size: Integer giving the batch size of noise to generate.
    - dim: Integer giving the dimension of noise to generate.

    Output:
    - A PyTorch Tensor of shape (batch_size, dim, 1, 1) containing uniform
      random noise in the range (-1, 1).
    """
    return to_var(torch.rand(batch_size, dim) * 2 - 1).unsqueeze(2).unsqueeze(3)
  

def upconv(in_channels, out_channels, kernel_size, stride=2, padding=2, batch_norm=True, spectral_norm=False):
    """Creates a upsample-and-convolution layer, with optional batch normalization.
    """
    layers = []
    if stride>1:
        layers.append(nn.Upsample(scale_factor=stride))
    conv_layer = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=1, padding=padding, bias=False)
    if spectral_norm:
        layers.append(SpectralNorm(conv_layer))
    else:
        layers.append(conv_layer)
    if batch_norm:
        layers.append(nn.BatchNorm2d(out_channels))
    return nn.Sequential(*layers)


def conv(in_channels, out_channels, kernel_size, stride=2, padding=2, batch_norm=True, init_zero_weights=False, spectral_norm=False):
    """Creates a convolutional layer, with optional batch normalization.
    """
    layers = []
    conv_layer = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding, bias=False)
    if init_zero_weights:
        conv_layer.weight.data = torch.randn(out_channels, in_channels, kernel_size, kernel_size) * 0.001
            
    if spectral_norm:
        layers.append(SpectralNorm(conv_layer))
    else:
        layers.append(conv_layer)

    if batch_norm:
        layers.append(nn.BatchNorm2d(out_channels))
    return nn.Sequential(*layers)
  

class ResnetBlock(nn.Module):
    def __init__(self, conv_dim):
        super(ResnetBlock, self).__init__()
        self.conv_layer = conv(in_channels=conv_dim, out_channels=conv_dim, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        out = x + self.conv_layer(x)
        return out

## DCGAN

## Spectral Norm class

### GAN generator

In [6]:
class DCGenerator(nn.Module):
    def __init__(self, noise_size, conv_dim, spectral_norm=False):
        super(DCGenerator, self).__init__()

        self.conv_dim = conv_dim
        self.relu = nn.ReLU()
        self.linear_bn = upconv(100, conv_dim*4,3) #BS X noise_size x 1 x 1 -> BS x 128 x 4 x 4 
        self.upconv1 = upconv(conv_dim*4,conv_dim*2,5)
        self.upconv2 = upconv(conv_dim*2,conv_dim,5)
        self.upconv3 = upconv(conv_dim,1,5, batch_norm=False) 
      
        self.tanh = nn.Tanh()

    def forward(self, z):
        """Generates an image given a sample of random noise.

            Input
            -----
                z: BS x noise_size x 1 x 1   -->  BSx100x1x1 (during training)

            Output
            ------
                out: BS x channels x image_width x image_height  -->  BSx3x32x32 (during training)
        """
        batch_size = z.size(0)
        out = self.relu(self.linear_bn(z))  # BS x 128 x 4 x 4      conv_dim=32
        out = out.view(-1, self.conv_dim*4, 4, 4)
        out = self.relu(self.upconv1(out))  # BS x 64 x 8 x 8
        out = self.relu(self.upconv2(out))  # BS x 32 x 16 x 16
        out = self.tanh(self.upconv3(out))  # BS x 3 x 32 x 32
        out_size = out.size()
        if out_size != torch.Size([batch_size, 1, 32, 32]):
            raise ValueError("expect {} x 3 x 32 x 32, but get {}".format(batch_size, out_size))
        return out


### GAN discriminator

In [7]:
class DCDiscriminator(nn.Module):
    """Defines the architecture of the discriminator network.
       Note: Both discriminators D_X and D_Y have the same architecture in this assignment.
    """
    def __init__(self, conv_dim=64, spectral_norm=False):
        super(DCDiscriminator, self).__init__()

        self.conv1 = conv(in_channels=1, out_channels=conv_dim, kernel_size=5, stride=2, spectral_norm=spectral_norm)
        self.conv2 = conv(in_channels=conv_dim, out_channels=conv_dim*2, kernel_size=5, stride=2, spectral_norm=spectral_norm)
        self.conv3 = conv(in_channels=conv_dim*2, out_channels=conv_dim*4, kernel_size=5, stride=2, spectral_norm=spectral_norm)
        self.conv4 = conv(in_channels=conv_dim*4, out_channels=1, kernel_size=5, stride=2, padding=1, batch_norm=False, spectral_norm=spectral_norm)

    def forward(self, x):
        batch_size = x.size(0)

        out = F.relu(self.conv1(x))    # BS x 64 x 16 x 16
        out = F.relu(self.conv2(out))    # BS x 64 x 8 x 8
        out = F.relu(self.conv3(out))    # BS x 64 x 4 x 4

        out = self.conv4(out).squeeze()
        out_size = out.size()
        if out_size != torch.Size([batch_size,]):
            raise ValueError("expect {} x 1, but get {}".format(batch_size, out_size))
        return out

### GAN training loop

In [8]:
def gan_training_loop(dataloader, test_dataloader, opts):
    """Runs the training loop.
        * Saves checkpoint every opts.checkpoint_every iterations
        * Saves generated samples every opts.sample_every iterations
    """

    # Create generators and discriminators
    G, D = create_model(opts)

    g_params = G.parameters()  # Get generator parameters
    d_params = D.parameters()  # Get discriminator parameters

    # Create optimizers for the generators and discriminators
    g_optimizer = optim.RMSprop(g_params, opts.lr)
    d_optimizer = optim.RMSprop(d_params, opts.lr)

    train_iter = iter(dataloader)

    test_iter = iter(test_dataloader)
    # Get some fixed data from domains X and Y for sampling. These are images that are held
    # constant throughout training, that allow us to inspect the model's performance.
    fixed_noise = sample_noise(100, opts.noise_size)  # # 100 x noise_size x 1 x 1

    iter_per_epoch = len(train_iter)
    total_train_iters = opts.train_iters

    losses = {"iteration": [], "D_fake_loss": [], "D_real_loss": [], "G_loss": [], "D_loss": []}

    # adversarial_loss = torch.nn.BCEWithLogitsLoss()
    gp_weight = 1
    epoch = 0
    total_iters = 0
    try:
        for iteration in range(1, opts.train_iters + 1):

            # Reset data_iter for each epoch
            
            # ones = Variable(torch.Tensor(real_images.shape[0]).float().cuda().fill_(1.0), requires_grad=False)
            if total_iters % iter_per_epoch == 0:
                    epoch +=1
                    train_iter = iter(dataloader)
                    print("EPOCH:", epoch)
            b = opts.batch_size
            for i in range(opts.n_critic):
                real_images, real_labels = train_iter.next()
                real_images, real_labels = to_var(real_images), to_var(real_labels).long().squeeze()
                m = b
                noise = sample_noise(m, opts.noise_size)
                fake_images = G(noise)
                D_real_loss = D(real_images).mean()
                D_fake_loss = D(fake_images).mean()

                D_loss = -(D_real_loss - D_fake_loss) #Minimize D_real_loss - D_fake_loss
                D_loss.backward()
                d_optimizer.step()

                Wasserstein_Distance = D_real_loss - D_fake_loss

                total_iters += 1    
                
                for param in D.parameters():
                    param.data.clamp_(-opts.clip, opts.clip)
                D.zero_grad()
                G.zero_grad()
            # 
            z = sample_noise(m, opts.noise_size)
            G_z = G(z)
            G_loss = -torch.mean(D(G_z))

            G_loss.backward()
            g_optimizer.step()
            D.zero_grad()
            G.zero_grad()

            if iteration % opts.log_step == 0:
                losses['iteration'].append(iteration)
                losses['D_real_loss'].append(D_real_loss.item())
                losses['D_loss'].append(D_real_loss.item())
          
                losses['D_fake_loss'].append(D_fake_loss.item())
                losses['G_loss'].append(G_loss.item())
                print('Iteration [{:4d}/{:4d}] | D_real_loss: {:6.4f} | D_fake_loss: {:6.4f} | G_loss: {:6.4f} | D_loss: {:6.4f} | Wasserstein_Distance: {:6.4f}'.format(
                    iteration, total_train_iters, D_real_loss.item(), D_fake_loss.item(), G_loss.item(), D_loss.item(), Wasserstein_Distance.item() ))

                

            # Save the generated samples
            if iteration % opts.sample_every == 0:
                gan_save_samples(G, fixed_noise, iteration, opts)

            # Save the model parameters
            if iteration % opts.checkpoint_every == 0:
                gan_checkpoint(iteration, G, D, opts)

    except KeyboardInterrupt:
        print('Exiting early from training.')
        return G, D

    plt.figure()
    plt.plot(losses['iteration'], losses['D_real_loss'], label='D_real')
    plt.plot(losses['iteration'], losses['D_fake_loss'], label='D_fake')
    plt.plot(losses['iteration'], losses['G_loss'], label='G')
    plt.plot(losses['iteration'], losses['D_loss'], label='D')

    plt.legend()
    plt.savefig(os.path.join(opts.sample_dir, 'losses.png'))
    plt.close()
    return G, D

# Training


## Download dataset

In [9]:
######################################################################
# Download Translation datasets
######################################################################
data_fpath = get_file(fname='emojis', 
                         origin='http://www.cs.toronto.edu/~jba/emojis.tar.gz', 
                         untar=True)

data/emojis.tar.gz


### WGAN

In [10]:
SEED = 11

# Set the random seed manually for reproducibility.
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)


args = AttrDict()
args_dict = {
              'clip': .01,
              'n_critic': 5,
              'image_size':32, 
              'g_conv_dim':32, 
              'd_conv_dim':64,
              'noise_size':100,
              'num_workers': 0,
              'train_iters':200000,
              'X':'letters',  # options: 'Windows' / 'Apple'
              'Y': None,
              'lr':5e-5,
              'beta1':0.5,
              'beta2':0.999,
              'batch_size':64, 
              'checkpoint_dir': 'results/checkpoints_gan_gp1_lr3e-5',
              'sample_dir': 'results/samples_gan_gp1_lr3e-5',
              'load': None,
              'log_step':200,
              'sample_every':200,
              'checkpoint_every':1000,
              'spectral_norm': False,
              'gradient_penalty': False,
              'd_train_iters': 1
}
args.update(args_dict)

print_opts(args)
G, D = train(args)

generate_gif("results/samples_gan_gp1_lr3e-5")

                                      Opts                                      
--------------------------------------------------------------------------------
                                   clip: 0.01                                   
                               n_critic: 5                                      
                             image_size: 32                                     
                             g_conv_dim: 32                                     
                             d_conv_dim: 64                                     
                             noise_size: 100                                    
                            train_iters: 200000                                 
                                      X: letters                                
                                     lr: 5e-05                                  
                                  beta1: 0.5                                    
                            

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:285: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


Models moved to GPU.
EPOCH: 1


Iteration [ 200/200000] | D_real_loss: 0.3591 | D_fake_loss: -0.3508 | G_loss: 0.3525 | D_loss: -0.7099 | Wasserstein_Distance: 0.7099
Saved results/samples_gan_gp1_lr3e-5/sample-000200.png
EPOCH: 2


Iteration [ 400/200000] | D_real_loss: 0.3822 | D_fake_loss: -0.3766 | G_loss: 0.3783 | D_loss: -0.7588 | Wasserstein_Distance: 0.7588
Saved results/samples_gan_gp1_lr3e-5/sample-000400.png


Iteration [ 600/200000] | D_real_loss: 0.3820 | D_fake_loss: -0.3761 | G_loss: 0.3766 | D_loss: -0.7581 | Wasserstein_Distance: 0.7581
Saved results/samples_gan_gp1_lr3e-5/sample-000600.png
EPOCH: 3


Iteration [ 800/200000] | D_real_loss: 0.3774 | D_fake_loss: -0.3725 | G_loss: 0.3765 | D_loss: -0.7499 | Wasserstein_Distance: 0.7499
Saved results/samples_gan_gp1_lr3e-5/sample-000800.png


Iteration [1000/200000] | D_real_loss: 0.3820 | D_fake_loss: -0.3769 | G_loss: 0.3771 | D_loss: -0.7589 | Wasserstein_Distance: 0.7589
Saved results/samples_gan_gp1_lr3e-5/sample-001000.png
EPOCH: 4


Iteration [1200/200000] | D_real_loss: 0.3834 | D_fake_loss: -0.3783 | G_loss: 0.3787 | D_loss: -0.7618 | Wasserstein_Distance: 0.7618
Saved results/samples_gan_gp1_lr3e-5/sample-001200.png


Iteration [1400/200000] | D_real_loss: 0.3815 | D_fake_loss: -0.3814 | G_loss: 0.3790 | D_loss: -0.7629 | Wasserstein_Distance: 0.7629
Saved results/samples_gan_gp1_lr3e-5/sample-001400.png
Exiting early from training.
adding image results/samples_gan_gp1_lr3e-5/sample-000200.png
adding image results/samples_gan_gp1_lr3e-5/sample-000400.png
adding image results/samples_gan_gp1_lr3e-5/sample-000600.png
adding image results/samples_gan_gp1_lr3e-5/sample-000800.png
adding image results/samples_gan_gp1_lr3e-5/sample-001000.png
adding image results/samples_gan_gp1_lr3e-5/sample-001200.png
adding image results/samples_gan_gp1_lr3e-5/sample-001400.png
adding image results/samples_gan_gp1_lr3e-5/sample-001600.png
adding image results/samples_gan_gp1_lr3e-5/sample-001800.png
adding image results/samples_gan_gp1_lr3e-5/sample-002000.png
adding image results/samples_gan_gp1_lr3e-5/sample-002200.png
adding image results/samples_gan_gp1_lr3e-5/sample-002400.png
adding image results/samples_gan_gp1_

In [ ]:
from google.colab import drive
drive.mount('/content/drive')